In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
%matplotlib inline

In [3]:
missing_value=["?"]
data=pd.read_csv('.\soybean-large.data',names=['Classes','date','plant-stand','precip','temp','hail','crop-hist','area-damaged','severity','seed-tmt','germination','plant-growth','leaves','leafspots-halo','leafspots-marg','leafspot-size','leaf-shread','leaf-malf','leaf-mild','stem','lodging','stem-cankers','canker-lesion','fruiting-bodies','external decay','mycelium','int-discolor','sclerotia','fruit-pods','fruit spots','seed','mold-growth','seed-discolor','seed-size','shriveling','roots'],na_values=missing_value)
data.head()

FileNotFoundError: [Errno 2] No such file or directory: '.\\soybean-large.data'

In [ ]:
from sklearn.impute import SimpleImputer
impute_Cat = SimpleImputer(missing_values=np.nan, strategy="most_frequent" )
data.iloc[:,1:] =impute_Cat.fit_transform(data.iloc[:,1:])
data

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder= LabelEncoder()
target=encoder.fit_transform(data["Classes"])

In [ ]:
features= data.drop("Classes",axis='columns')

In [ ]:
#splitting into tarin test
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(
    features, target, test_size=0.2, random_state=42)

In [ ]:
#MOdeling
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=50)
model.fit(xtrain, ytrain)

In [ ]:
#Evaluation
from sklearn.metrics import confusion_matrix, accuracy_score, plot_confusion_matrix
plot_confusion_matrix (model, xtest, ytest)

In [ ]:
ypred = model.predict(xtest)
confusion_matrix(ytest, ypred)

In [ ]:
from sklearn.model_selection import cross_val_score

cv = cross_val_score(model, features, target, cv=10)
cv
cv.mean()

In [ ]:
accuracy_score(ypred, ytest)

In [ ]:
model.get_params()

In [ ]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(
    model, 
    param_grid={
        "criterion": ["gini", "entropy"],
        "max_features": ["sqrt", "auto"],
        "min_samples_split": [2,3,4,5,6,7,8,9,11, 15, 20, 25]
    }, cv = 8, scoring="accuracy", verbose=0
)
grid.fit(features, target)

In [ ]:
grid.best_estimator_

In [ ]:
grid.best_params_

In [ ]:
bmodel = grid.best_estimator_
bmodel.fit(features["date"], target)
# Visualization

xmin, xmax = features["date"][:, 0].min(), target[:, 0].max()
ymin, ymax = features["date"][:, 1].min(), target[:, 1].max()

xx, yy = np.meshgrid(np.arange(xmin, xmax, 0.01), np.arange(ymin, ymax, 0.01))
xx.shape, yy.shape

xtest = np.c_[xx, yy]
xtest.shape
ypred = model.predict(xtest)
ypred = ypred.reshape(yy.shape)
ypred

plt.figure(1, figsize=(20,8))
plt.set_cmap(plt.cm.Accent_r)
plt.pcolormesh(xx, yy, ypred, shading='auto')

plt.scatter(features["date"][:, 0], features["date"][:, 1], c=target, edgecolors='black')
plt.xlim(xx.min(), xx.max())
plt.ylim(yy.min(), yy.max())
plt.show()